In [6]:
import pandas as pd
import re
import math

## 01. 計算 t期 企業互惠 & indegree & outdegree

In [14]:
for t_period in range(12, 18):
    
    counting_result = {}
    reciprocity_companies = {}
    counting_data = pd.read_excel(f"dyad_release_{t_period}.xlsx")
    counting_data.drop_duplicates()

    all_company_in_t = set(list(counting_data['supporting member']) + list(counting_data['rapporteur']))
    
    for i in all_company_in_t:
        counting_result[i] = {'outdegree':0,'indegree':0, 'reciprocity':0, 'sss': 0}
        reciprocity_companies[i] = []
        
    print(f'{len(all_company_in_t)} companies in release {t_period}')
    
    
    ###### 計算t期的 indegree & outdegree ###### 
    
    for index, row in counting_data.iterrows():
        if row['rapporteur'] != row['supporting member']:
            counting_result[row['rapporteur']]['indegree'] += 1
            counting_result[row['supporting member']]['outdegree'] += 1
            reciprocity_companies[row['rapporteur']] += [row['supporting member']]

        else:
            counting_result[row['rapporteur']]['sss'] += 1

        
    ###### 計算t期的 reciprocity ###### 
    
    for rapporteur, supporters in reciprocity_companies.items():
        for supporter in set(supporters):
            
            # 如果這些supporter也有擔任過WIR
            
            if supporter in reciprocity_companies and supporter != rapporteur:
                if rapporteur in reciprocity_companies[supporter]:
                    counting_result[rapporteur]['reciprocity'] += 1
                    
                    
    ###### t期的 indegree & outdegree 扣除 reciprocity ######
    
    for key, value in counting_result.items():
        value['outdegree'] = value['outdegree'] - value['reciprocity']
        value['indegree'] = value['indegree'] - value['reciprocity']
    
    print(counting_result['huawei'])
    
    
    ###### save file ######
    
    t_data = pd.DataFrame.from_dict(counting_result, orient='index').rename(columns={'outdegree':'pre-out','indegree':'pre-in','reciprocity':'pre-reci'})
    t_data.to_excel(f'reciprocity_rel{t_period}_0928-1.xlsx', index=True)

197 companies in release 12
{'outdegree': 363, 'indegree': 1258, 'reciprocity': 31, 'sss': 110}
200 companies in release 13
{'outdegree': 327, 'indegree': 828, 'reciprocity': 27, 'sss': 131}
222 companies in release 14
{'outdegree': 202, 'indegree': 864, 'reciprocity': 25, 'sss': 65}
248 companies in release 15
{'outdegree': 161, 'indegree': 1024, 'reciprocity': 21, 'sss': 88}
281 companies in release 16
{'outdegree': 359, 'indegree': 1286, 'reciprocity': 25, 'sss': 113}
298 companies in release 17
{'outdegree': 447, 'indegree': 1316, 'reciprocity': 32, 'sss': 122}


## 02. 整合t期indegree & outdegree資料與t+1期WIR、supporting次數

In [15]:
complete_data = pd.DataFrame()

for period_t_plus_1 in range(13, 19):
    
    
    ###### t+1期 擔任WIR, SUP次數 ######

    t_plus_1_data = pd.read_excel(f"company_overview_new.xlsx")[['company', f'rap-{period_t_plus_1}', f'sup-{period_t_plus_1}']].rename(columns={f'rap-{period_t_plus_1}': 'wir', f'sup-{period_t_plus_1}': 'sup'})
    t_plus_1_data.dropna(inplace=True)  
    t_plus_1_data.insert(1, 'release', period_t_plus_1)
    
    
    ###### t期 indegree, outdegree, 互惠次數, SSS次數 ######
    
    period_t = period_t_plus_1 - 1
    t_data = pd.read_excel(f'reciprocity_rel{period_t}_0928-1.xlsx').rename(columns={f'Unnamed: 0': 'company'})
    t_data.insert(1, 'release', period_t_plus_1)
    
    ###### 合併 t期, t+1期 數據 ######
    t_plus_1_data = pd.merge(t_plus_1_data,t_data,on=['company', 'release'], how='outer')
    complete_data = pd.concat([complete_data, t_plus_1_data])
    
    complete_data['W/(W+S)'] = complete_data['wir'] / (complete_data['wir'] + complete_data['sup'])
    
complete_data.fillna(0, inplace=True)
complete_data.to_excel(f'relation_wir_reciprocity7.xlsx', index=False)

## 03. 建立Dyad資料－依變數、自變數、控制變數

In [17]:
complete_data.head()

,company,release,wir,sup,pre-out,pre-in,pre-reci,sss,W/(W+S)
0,huawei,13,137.0,482.0,363.0,1258.0,31.0,110.0,0.221325
1,ericsson,13,107.0,509.0,447.0,1096.0,35.0,137.0,0.173701
2,nokia,13,67.0,495.0,492.0,445.0,30.0,45.0,0.119217
3,qualcomm,13,47.0,345.0,351.0,309.0,32.0,29.0,0.119898
4,china mobile,13,23.0,105.0,122.0,344.0,17.0,22.0,0.179688


In [23]:
total_db = pd.DataFrame()
wir_reciprocity = pd.read_excel(f"relation_wir_reciprocity7.xlsx")
company_data = pd.read_excel(f"company data.xlsx")
ldv_data = pd.read_excel(f"company_overview_new.xlsx")
sse_focal = {}

for rel in range(13, 19):
    cur_result = {}
    cur_company = list(wir_reciprocity[wir_reciprocity.release == rel]['company'])
    print(f'Log {rel} start...')
    
# 每一筆資料都要放大n-1倍
    for index, row in wir_reciprocity[wir_reciprocity.release == rel].iterrows():

        
        ###### t期 SSE ######
        
        if rel == 13:
            condition_before_13 = (wir_reciprocity.company == row['company']) & (wir_reciprocity.release == 13)
            sse_in_12 = wir_reciprocity[condition_before_13][['pre-out','pre-in']].sum(axis=1).item()
            
            if sse_in_12 != 0:
                sse_focal[row['company']] = 1
            else:
                sse_focal[row['company']] = 0
        
        else:
            if row['company'] not in sse_focal:
                sse_focal[row['company']] = 0
                
            condition = (wir_reciprocity.company == row['company']) & (wir_reciprocity.release == (rel-1))

            try:
                sse_in_pre = wir_reciprocity[condition][['wir','sup']].sum(axis=1).item()
            except:
                sse_focal[row['company']] += 0
            else:
                if sse_in_pre != 0:
                    sse_focal[row['company']] += 1

#         print(sse_focal[row['company']])


        ###### t期 LDV ######

        try:
            pre_Y1 = int(ldv_data[ldv_data.company == row['company']][f'rap-{rel-1}'].item())
        except:
            pre_Y1 = 0
        
        try:
            pre_Y2 = int(ldv_data[ldv_data.company == row['company']][f'sup-{rel-1}'].item())
        except:
            pre_Y2 = 0
    
        try:
            pre_Y3 = pre_Y1 / (pre_Y1 + pre_Y2)
        except:
            pre_Y3 = 0
            
            
        
        for partner in cur_company:
            if partner != row['company']:

                
                ###### NATION: focal & partner ######
                
                nat_foc = company_data[company_data['公司'] == row['company']]['NAT'].item()
                nat_part = company_data[company_data['公司'] == partner]['NAT'].item()
                if nat_foc != 5:
                    if nat_foc == nat_part:
                        is_nat_same = 1
                    else:
                        is_nat_same = 0
                else:
                    is_nat_same = 0
                    
                
                condition = (wir_reciprocity.company == partner) & (wir_reciprocity.release == rel)
                
                cur_result[f"{row['company']}, {partner}"] = {
                    'release': rel,
                    'Y1': row['wir'],
                    'Y2': row['sup'],
                    'Y3': row['W/(W+S)'],
                    'fre': row['wir'] + row['sup'],
                    'X1': row['pre-reci'],
                    'X2': wir_reciprocity[condition]['pre-out'].item(),
                    'X3': wir_reciprocity[condition]['pre-in'].item(),
                    'X1(log)': math.log(1 if row['pre-reci'] == 0 else row['pre-reci']),
                    'X2(log)': math.log(1 if wir_reciprocity[condition]['pre-out'].item() == 0 else wir_reciprocity[condition]['pre-out'].item()),
                    'X3(log)': math.log(1 if wir_reciprocity[condition]['pre-in'].item() == 0 else wir_reciprocity[condition]['pre-in'].item()),
                    'REL(14)': 1 if rel == 14 else 0,
                    'REL(15)': 1 if rel == 15 else 0,
                    'REL(16)': 1 if rel == 16 else 0,
                    'REL(17)': 1 if rel == 17 else 0,
                    'REL(18)': 1 if rel == 18 else 0,
                    'LDV(Y1)':pre_Y1,
                    'LDV(Y2)':pre_Y2,
                    'LDV(Y3)':pre_Y3,
                    'TEG': 0 if rel == 13 or rel == 14 else 1,
                    'NAT(I)':nat_foc,
                    'NAT(D)': is_nat_same,
                    'TEF': company_data[company_data['公司'] == row['company']]['TEL'].item(),
                    'REP': row['pre-out'],
                    'STA': row['pre-in'],
                    'REP(log)': math.log(1 if row['pre-out'] == 0 else row['pre-out']),
                    'STA(log)': math.log(1 if row['pre-in'] == 0 else row['pre-in']),
                    'SSE': sse_focal[row['company']],
                    'SSS': row['sss'],
                }
        
    curent_db = pd.DataFrame.from_dict(cur_result, orient='index')
    total_db = pd.concat([total_db, curent_db])

Log 13 start...
Log 14 start...
Log 15 start...
Log 16 start...
Log 17 start...
Log 18 start...


In [1]:
# total_db

In [25]:
total_db.to_excel(f'variable_0927-2.xlsx', index=True)